# Diving into Pinecone

This notebook uses the latest versions of the Pinecone library.

In [ ]:
# pip install -q pinecone-client

In [ ]:
# pip install --upgrade -q pinecone-client

In [ ]:
# pip show pinecone-client

In [1]:
# authenticating to Pinecone. 
# the API KEY is in .env
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from pinecone import Pinecone, ServerlessSpec

# Initilizing and authenticating the pinecone client
pc = Pinecone()
# pc = Pinecone(api_key='YOUR_API_KEY')

# checking authentication
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-8897cf7.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## Working with Pinecone Indexes

In [3]:
# listing all indexes
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-8897cf7.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [4]:
index_name = 'langchain'
# getting a complete description of a specific index:
pc.describe_index(index_name)

{'dimension': 1536,
 'host': 'langchain-8897cf7.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [10]:
# getting a list with the index names 
pc.list_indexes().names()

['langchain']

In [8]:
# deleting an index
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name} ... ')
    pc.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Index langchain does not exist!


In [9]:
# creating a Serverless Pinecone index 
# starter free plan permits 1 project, up to 5 indexes, up to 100 namespaces per index
index_name = 'langchain'

if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print('Index created! 😊')
else:
    print(f'Index {index_name} already exists!')


Creating index langchain
Index created! 😊


In [11]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Working with Vectors

In [12]:
# inserting vectors
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_name = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [13]:
# updating vectors
index.upsert(vectors=[('c', [0.5] * 1536)])

{'upserted_count': 1}

In [14]:
# fetching vectors
# index = pc.Index(index_name)
index.fetch(ids=['c', 'd'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [15]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [17]:
# querying a non-existing vector returns an empty vector
index.fetch(ids=['x']) 

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [18]:
# querying vectors
query_vector = [random.random() for _ in range(1536)]

In [19]:
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'd', 'score': 0.762393892, 'values': []},
             {'id': 'a', 'score': 0.761265814, 'values': []},
             {'id': 'e', 'score': 0.758823, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Namespaces

In [20]:
# index.describe_index_stats()
index = pc.Index('langchain')

import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [21]:
# partition the index into namespaces
# creating a new namespace
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [22]:
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3}},
 'total_vector_count': 8}

In [24]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [25]:
index.fetch(ids=['x'], namespace='first-namespace')


{'namespace': 'first-namespace',
 'usage': {'read_units': 1},
 'vectors': {'x': {'id': 'x',
                   'values': [0.490776807,
                              0.856016219,
                              0.294744372,
                              0.35136047,
                              0.21365872,
                              0.809774518,
                              0.615277648,
                              0.932494342,
                              0.291038513,
                              0.858095229,
                              0.941571176,
                              0.181766763,
                              0.272200197,
                              0.835601389,
                              0.598871887,
                              0.0843337551,
                              0.415596068,
                              0.356585085,
                              0.579105198,
                              0.8078776,
                              0.731160164,
        

In [27]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [29]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [30]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 7}